In [1]:
from dashi_infer import ViTInfer
from torchvision.datasets import ImageFolder
import glob
from PIL import Image

DATA_PATH = "../dataset/synthetic_dataset_with_bg_alltools" 
full_image_folder = ImageFolder(root=DATA_PATH)
paths_and_labels = full_image_folder.samples # Список кортежей (путь, метка)
class_names = full_image_folder.classes

model = ViTInfer(
    # weights_path="best_vit_tools_model.pth",
    weights_path="best_vit_tools_model_6.pth",
    # weights_path="best_vit_tools_model_4.pth",
    # weights_path="best_vit_tools_model_5.pth",
    # model_name="vit_base_patch16_224",
    model_name="vit_small_patch16_224",
    class_names=class_names,
    # img_size=512,
    img_size=224,
    mean=(0.5, 0.5, 0.5),
    std=(0.5, 0.5, 0.5),
    device="cuda:1"
)

# Списки для хранения истинных и предсказанных меток
y_true = []
y_pred = []

# Предполагается, что full_image_folder.classes содержит список всех классов
for i in full_image_folder.classes:
    samples = glob.glob(f"../dataset/razmetka_cropped_new/{i}/*.jpg")
    for sample in samples:
        # Получаем предсказанный класс
        id, class_name, prob, probs = model.infer(Image.open(sample))
        
        # 'i' - это истинная метка
        y_true.append(i)
        # 'class_name' - это предсказанная метка
        y_pred.append(class_name)

from sklearn.metrics import f1_score, classification_report
import numpy as np # Может понадобиться для работы со списками

# Убедимся, что y_true и y_pred - это numpy массивы или списки
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# --- Расчёт F1-меры ---

# 1. Макро-F1: Простое среднее F1-мер для каждого класса (игнорируя дисбаланс классов)
# Предпочтительно, если вам важна равная производительность по всем классам
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"Макро-F1: {macro_f1:.4f}")

# 2. Взвешенная F1: Среднее F1-мер, взвешенное по количеству образцов в каждом классе
# Предпочтительно при дисбалансе классов
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"Взвешенная F1: {weighted_f1:.4f}")

# 3. Micro-F1 (Эквивалентно Accuracy при многоклассовой классификации)
# micro_f1 = f1_score(y_true, y_pred, average='micro')
# print(f"Микро-F1 (Accuracy): {micro_f1:.4f}")

# --- Дополнительные метрики (рекомендуется) ---

# Полный отчёт с Precision, Recall, F1-score для каждого класса и общими средними
report = classification_report(y_true, y_pred)
print("\nОтчет о классификации:\n", report)

/home/ubuntu/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Макро-F1: 0.8856
Взвешенная F1: 0.8848

Отчет о классификации:
                                  precision    recall  f1-score   support

                 1 Отвертка «-»       1.00      0.54      0.70        13
    10 Ключ рожковыйнакидной  ¾       0.76      1.00      0.87        13
                    11 Бокорезы       1.00      1.00      1.00        13
                 2 Отвертка «+»       0.60      0.92      0.73        13
  3 Отвертка на смещенный крест       0.83      0.77      0.80        13
                    4 Коловорот       0.93      1.00      0.96        13
       5 Пассатижи контровочные       0.92      1.00      0.96        12
                    6 Пассатижи       1.00      0.93      0.96        14
                      7 Шэрница       1.00      1.00      1.00        13
               8 Разводной ключ       1.00      0.62      0.76        13
9 Открывашка для банок с маслом       1.00      1.00      1.00        12

                       accuracy                           

In [6]:
from PIL import Image
for img in [
    "../dataset/dataset/1 Отвертка «-»/cropped/DSCN2385_cropped.JPG", 
    "../dataset/dataset/2 Отвертка «+»/cropped/DSCN1396_cropped.JPG",
    "../dataset/dataset/3 Отвертка на смещенный крест/cropped/DSCN1664_cropped.JPG",
    "./image_2025-09-28_19-28-50.png",
    "./image_2025-09-28_19-30-23.png",
    "./image_2025-09-28_19-30-33.png",
    "./image_2025-09-28_19-33-40.png",
    "./image_2025-09-28_20-18-38.png",
    "./image_2025-09-28_20-18-46.png",
    "./image_2025-09-28_20-18-58.png",
]:
    id, class_name, prob, probs = model.infer(Image.open(img))
    print(class_name, prob)

1 Отвертка «-» 0.9981550574302673
2 Отвертка «+» 0.9999682903289795
3 Отвертка на смещенный крест 0.9959751963615417
10 Ключ рожковыйнакидной  ¾ 0.5328171253204346
6 Пассатижи 0.3902723491191864
2 Отвертка «+» 0.3289916217327118
2 Отвертка «+» 0.8722331523895264
3 Отвертка на смещенный крест 0.8438364863395691
3 Отвертка на смещенный крест 0.9955884218215942
3 Отвертка на смещенный крест 0.9984765648841858


In [ ]:
model.infer()

In [5]:
len(paths_and_labels)

20099

In [8]:
int(len(paths_and_labels)*0.5)

10049

In [ ]:
import random

random.sample(paths_and_labels, int(len(paths_and_labels)*0.5))

[('../dataset/synthetic_dataset_with_bg_alltools/1 Отвертка «-»/DSCN2431_masked_scene_2_with_bg.jpg',
  0),
 ('../dataset/synthetic_dataset_with_bg_alltools/11 Бокорезы/DSCN0886_masked_scene_1_with_bg.jpg',
  2),
 ('../dataset/synthetic_dataset_with_bg_alltools/2 Отвертка «+»/DSCN1406_masked_scene_3_with_bg.jpg',
  3),
 ('../dataset/synthetic_dataset_with_bg_alltools/5 Пассатижи контровочные/DSCN4067_masked_scene_2_with_bg.jpg',
  6),
 ('../dataset/synthetic_dataset_with_bg_alltools/10 Ключ рожковыйнакидной  ¾/DSCN2307 (2)_masked_scene_1_with_bg.jpg',
  1),
 ('../dataset/synthetic_dataset_with_bg_alltools/11 Бокорезы/DSCN0945_masked_scene_5_with_bg.jpg',
  2),
 ('../dataset/synthetic_dataset_with_bg_alltools/7 Шэрница/DSCN0501_masked_scene_3_with_bg.jpg',
  8),
 ('../dataset/synthetic_dataset_with_bg_alltools/4 Коловорот/DSCN3650_masked_scene_4_with_bg.jpg',
  5),
 ('../dataset/synthetic_dataset_with_bg_alltools/5 Пассатижи контровочные/DSCN2199_masked_scene_2_with_bg.jpg',
  6),
 ('..

In [2]:
import torch
import torch.nn as nn
import timm
import numpy as np
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

# ======================================================================================
# 1. Параметры и Настройка
# ======================================================================================

# Параметры, использованные при обучении
IMG_SIZE = 512
NUM_CLASSES = len(class_names)
MEAN = [0.5, 0.5, 0.5]
STD = [0.5, 0.5, 0.5]
# MEAN = [0.485, 0.456, 0.406]
# STD = [0.229, 0.224, 0.225]
MODEL_PATH = 'best_vit_tools_model.pth' # Путь к сохраненным весам
# CLASS_NAMES = ["1 Отвертка «-»", "2 Отвертка «+»", "3 Отвертка на смещенный крест"]
CLASS_NAMES = class_names.copy()

# Устройство (CPU/GPU)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# ======================================================================================
# 2. Трансформации для Инференса
# ======================================================================================

# Использование тех же трансформаций, что и для валидации/теста
# ВАЖНО: Только изменение размера и нормализация
inference_transforms = A.Compose([
    A.LongestMaxSize(max_size=IMG_SIZE),
    A.PadIfNeeded(min_height=IMG_SIZE, min_width=IMG_SIZE, border_mode=cv2.BORDER_CONSTANT, fill=(0, 0, 0)),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

# ======================================================================================
# 3. Инициализация и Загрузка Модели
# ======================================================================================

def load_model(model_path, num_classes, img_size, device):
    """Инициализация модели ViT и загрузка обученных весов."""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Файл весов не найден: '{model_path}'. Убедитесь, что вы запустили обучение.")

    # Создание той же архитектуры, что и при обучении
    model = timm.create_model(
        'vit_base_patch16_224',
        pretrained=False, # Не загружаем ImageNet, будем использовать свои веса
        num_classes=num_classes,
        img_size=img_size
    )
    
    # Загрузка сохраненных весов
    # map_location='cpu' гарантирует, что веса загружаются корректно,
    # даже если GPU недоступен, а затем переносим на нужное устройство
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.to(device)
    model.eval() # Установка модели в режим оценки (ВАЖНО!)
    
    print(f"Модель ViT успешно загружена из '{model_path}' и переведена в режим eval.")
    return model

# Загружаем модель
model = load_model(MODEL_PATH, NUM_CLASSES, IMG_SIZE, device)

# ======================================================================================
# 4. Функция Инференса
# ======================================================================================


/home/ubuntu/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Модель ViT успешно загружена из 'best_vit_tools_model.pth' и переведена в режим eval.


In [5]:

@torch.no_grad()
def predict_image(pil_image: Image.Image, model: nn.Module, transforms: A.Compose, class_names: list, device: torch.device) -> str:
    """
    Выполняет предсказание класса для одного изображения.

    :param pil_image: Входное изображение в формате PIL.Image.
    :param model: Загруженная модель PyTorch.
    :param transforms: Объект трансформаций Albumentations.
    :param class_names: Список имен классов.
    :param device: Устройство (CPU/GPU).
    :return: Имя предсказанного класса (строка).
    """
    # 1. Преобразование PIL в NumPy (RGB) для Albumentations
    image_np = np.array(pil_image.convert("RGB"))
    
    # 2. Применение трансформаций
    augmented = transforms(image=image_np)
    # Получаем PyTorch Tensor (C, H, W)
    image_tensor = augmented["image"] 
    
    # 3. Добавление размерности батча (N, C, H, W) и перенос на устройство
    input_tensor = image_tensor.unsqueeze(0).to(device)
    
    # 4. Прямой проход (Forward pass)
    outputs = model(input_tensor)
    
    # 5. Получение предсказанного класса
    # Применяем softmax для преобразования логитов в вероятности
    probabilities = torch.softmax(outputs, dim=1)
    # print(probabilities)
    # Находим индекс максимальной вероятности
    predicted_index = torch.argmax(probabilities, dim=1).item()
    
    # 6. Получение имени класса
    predicted_class = class_names[predicted_index]
    
    # Для отладки:
    max_prob = probabilities[0, predicted_index].item()
    print(f"Предсказанный индекс: {predicted_index}, Вероятность: {max_prob:.4f}")

    return predicted_index, predicted_class

In [8]:

# ======================================================================================
# 5. Пример Использования
# ======================================================================================

# Шаг 1: Создайте или загрузите изображение для теста
# Здесь используется заглушка. В реальном коде замените на:
# test_image = Image.open("path/to/your/test_image.jpg")
for img in [
    "../dataset/dataset/1 Отвертка «-»/cropped/DSCN2385_cropped.JPG", 
    "../dataset/dataset/2 Отвертка «+»/cropped/DSCN1396_cropped.JPG",
    "../dataset/dataset/3 Отвертка на смещенный крест/cropped/DSCN1664_cropped.JPG",
    "./image_2025-09-28_19-28-50.png",
    "./image_2025-09-28_19-30-23.png",
    "./image_2025-09-28_19-30-33.png",
    "./image_2025-09-28_19-33-40.png",
    "./image_2025-09-28_20-18-38.png",
    "./image_2025-09-28_20-18-46.png",
    "./image_2025-09-28_20-18-58.png",
]:
    try:
        # Создаем фиктивное изображение
        dummy_image = Image.open(img)
        
        # Шаг 2: Вызов функции предсказания
        predicted_index, predicted_label = predict_image(
            pil_image=dummy_image,
            model=model,
            transforms=inference_transforms,
            class_names=CLASS_NAMES,
            device=device
        )

        # Шаг 3: Вывод результата
        print("file: ", img)
        print(f"✨ Предсказанный класс: {predicted_label} \n")

    except FileNotFoundError as e:
        print(f"Ошибка: {e}")
        print("Пожалуйста, убедитесь, что файл 'best_vit_screwdriver_model.pth' существует.")

    except Exception as e:
        print(f"Произошла ошибка при инференсе: {e}")

Предсказанный индекс: 0, Вероятность: 0.9982
file:  ../dataset/dataset/1 Отвертка «-»/cropped/DSCN2385_cropped.JPG
✨ Предсказанный класс: 1 Отвертка «-» 

Предсказанный индекс: 3, Вероятность: 1.0000
file:  ../dataset/dataset/2 Отвертка «+»/cropped/DSCN1396_cropped.JPG
✨ Предсказанный класс: 2 Отвертка «+» 

Предсказанный индекс: 4, Вероятность: 0.9960
file:  ../dataset/dataset/3 Отвертка на смещенный крест/cropped/DSCN1664_cropped.JPG
✨ Предсказанный класс: 3 Отвертка на смещенный крест 

Предсказанный индекс: 1, Вероятность: 0.5328
file:  ./image_2025-09-28_19-28-50.png
✨ Предсказанный класс: 10 Ключ рожковыйнакидной  ¾ 

Предсказанный индекс: 7, Вероятность: 0.3903
file:  ./image_2025-09-28_19-30-23.png
✨ Предсказанный класс: 6 Пассатижи 

Предсказанный индекс: 3, Вероятность: 0.3290
file:  ./image_2025-09-28_19-30-33.png
✨ Предсказанный класс: 2 Отвертка «+» 

Предсказанный индекс: 3, Вероятность: 0.8722
file:  ./image_2025-09-28_19-33-40.png
✨ Предсказанный класс: 2 Отвертка «+» 


In [11]:
import glob
from sklearn.metrics import classification_report

DATA_PATTERNS = [
    "../dataset/dataset/1 Отвертка «-»/cropped/*.JPG",
    "../dataset/dataset/2 Отвертка «+»/cropped/*.JPG",
    "../dataset/dataset/3 Отвертка на смещенный крест/cropped/*.JPG",
]

def evaluate_predictions(model, data_patterns, class_names, transforms, device):
    """
    Выполняет массовый прогноз для всех файлов и считает метрики.
    """
    true_labels = []    # Фактические метки (индексы)
    predictions = []    # Предсказанные метки (индексы)
    
    # Создаем словарь для маппинга имени папки к индексу класса
    # Например: "1 Отвертка «-»" -> 0
    class_to_idx = {name: i for i, name in enumerate(class_names)}

    print("\n--- Запуск массового прогноза ---")
    
    # Итерируемся по каждому паттерну (папке класса)
    for pattern in data_patterns:
        # Извлекаем имя класса из пути (например, "1 Отвертка «-»")
        class_name_part = pattern.split('/')[-3]
        
        # Получаем фактический индекс класса
        try:
            true_label_index = class_to_idx[class_name_part]
        except KeyError:
             print(f"Внимание: Класс '{class_name_part}' не найден в CLASS_NAMES. Пропускаю.")
             continue

        file_list = glob.glob(pattern)
        if not file_list:
            print(f"Предупреждение: Файлы не найдены по паттерну: {pattern}")
            continue

        print(f"Обработка {len(file_list)} изображений для класса: {class_name_part}")
        
        for file_path in file_list:
            try:
                # Открываем изображение
                img = Image.open(file_path)
                
                # Получаем предсказанный индекс
                predicted_index, predicted_class = predict_image(img, model, transforms, class_names, device)
                
                # Сохраняем истинный и предсказанный индекс
                true_labels.append(true_label_index)
                predictions.append(predicted_index)

            except Exception as e:
                print(f"Ошибка обработки файла {file_path}: {e}")
                
    
    # Расчет метрик только если есть данные
    if not true_labels:
        print("Нет данных для расчета метрик.")
        return

    print("\n--- Отчет по Классификации ---")
    print(classification_report(true_labels, predictions, target_names=class_names, zero_division=0))

# ======================================================================================
# 6. Запуск Оценки
# ======================================================================================

# Вызываем функцию для массового прогноза и расчета метрик
evaluate_predictions(model, DATA_PATTERNS, CLASS_NAMES, inference_transforms, device)



--- Запуск массового прогноза ---
Обработка 247 изображений для класса: 1 Отвертка «-»
Предсказанный индекс: 0, Вероятность: 0.9999
Предсказанный индекс: 0, Вероятность: 0.9999
Предсказанный индекс: 0, Вероятность: 0.9998
Предсказанный индекс: 0, Вероятность: 0.9984
Предсказанный индекс: 0, Вероятность: 0.9989
Предсказанный индекс: 0, Вероятность: 0.9979
Предсказанный индекс: 0, Вероятность: 0.9947
Предсказанный индекс: 0, Вероятность: 0.9993
Предсказанный индекс: 0, Вероятность: 0.9982
Предсказанный индекс: 0, Вероятность: 0.9999
Предсказанный индекс: 0, Вероятность: 0.9997
Предсказанный индекс: 0, Вероятность: 1.0000
Предсказанный индекс: 0, Вероятность: 0.9979
Предсказанный индекс: 0, Вероятность: 0.8054
Предсказанный индекс: 0, Вероятность: 0.9975
Предсказанный индекс: 0, Вероятность: 0.9986
Предсказанный индекс: 0, Вероятность: 0.9975
Предсказанный индекс: 0, Вероятность: 0.7997
Предсказанный индекс: 0, Вероятность: 0.9898
Предсказанный индекс: 0, Вероятность: 0.9291
Предсказанны

ValueError: Number of classes, 3, does not match size of target_names, 11. Try specifying the labels parameter

In [10]:
CLASS_NAMES

['1 Отвертка «-»',
 '10 Ключ рожковыйнакидной  ¾',
 '11 Бокорезы',
 '2 Отвертка «+»',
 '3 Отвертка на смещенный крест',
 '4 Коловорот',
 '5 Пассатижи контровочные',
 '6 Пассатижи',
 '7 Шэрница',
 '8 Разводной ключ',
 '9 Открывашка для банок с маслом']